In [1]:
import transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM
import datasets
from datasets import load_dataset

import sys
import src.evals.data as data_module

import torch

import random

/home/azureuser/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = "answerdotai/ModernBERT-large"

In [3]:
chat = data_module.create_preference_to_flan_style_dataset(
    task = "sarahpann/rwb_chat",
    split = "train",
    tokenizer_name = tokenizer,
    max_seq_length = 3600,
    prefix = "Which response is the most helpful, relevant, and correct? ",

    dataset_name="sarahpann/rwb_chat",
    dataset_subset="",
    task_column_names={"sarahpann/rwb_chat": ('chosen', 'rejected', 'og_dataset')}
)

chat_hard = data_module.create_preference_to_flan_style_dataset(
    task = "sarahpann/rwb_chat_hard",
    split = "train",
    tokenizer_name = tokenizer,
    max_seq_length = 3600,
    prefix = "Which response is the most helpful, relevant, and correct? ",

    dataset_name="sarahpann/rwb_chat_hard",
    dataset_subset="",
    task_column_names={"sarahpann/rwb_chat_hard": ('chosen', 'rejected', 'og_dataset')}
)

reasoning = data_module.create_preference_to_flan_style_dataset(
    task="sarahpann/rwb_reasoning",
    split='train',
    tokenizer_name=tokenizer,
    max_seq_length=3600,
    prefix="Determine which response is the best choice based on mathematical or programming accuracy. ",

    dataset_name="sarahpann/rwb_reasoning",
    dataset_subset="",
    task_column_names={"sarahpann/rwb_reasoning": ('chosen', 'rejected', 'og_dataset')}
)

safety = data_module.create_preference_to_flan_style_dataset(
    task="sarahpann/rwb_safety",
    split='train',
    tokenizer_name=tokenizer,
    max_seq_length=3600,
    prefix="Which response is the most helpful, relevant, and correct? ",

    dataset_name="sarahpann/rwb_safety",
    dataset_subset="",
    task_column_names={"sarahpann/rwb_safety": ('chosen', 'rejected', 'og_dataset')}
)

Map: 100%|██████████| 740/740 [00:00<00:00, 4844.13 examples/s]


In [4]:
model = AutoModelForMaskedLM.from_pretrained("sarahpann/reasoning_model_large")

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading adapter weights from sarahpann/reasoning_model_large led to missing keys in the model: model.layers.0.attn.Wqkv.lora_A.default.weight, model.layers.0.attn.Wqkv.lora_B.default.weight, model.layers.0.attn.Wqkv.lora_magnitude_vector.default.weight, model.layers.0.attn.Wo.lora_A.default.weight, model.layers.0.attn.Wo.lora_B.default.weight, model.layers.0.attn.Wo.lora_magnitude_vector.default.weight, model.layers.0.mlp.Wi.lora_A.default.weight, model.layers.0.mlp.Wi.lora_B.default.weight, model.layers.0.mlp.Wi.lora_magnitude_vector.default.weight, model.layers.0.mlp.Wo.lora_A.default.weight, model.layers.0.mlp.Wo.lora_B.default.weight, model.layers.0.mlp.Wo.lora_magnitude_vector.default.weight, model.layers.1.attn.Wqkv.lora_A.default.weight, model.layers.1.attn.Wqkv.lora_B.default.weight, model.layers.1.attn.Wqkv.

In [5]:
chat.set_format(type='torch', columns=['input_ids', 'attention_mask'])
chat_hard.set_format(type='torch', columns=['input_ids', 'attention_mask'])
reasoning.set_format(type='torch', columns=['input_ids', 'attention_mask'])
safety.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [6]:
chat_loader = torch.utils.data.DataLoader(chat, batch_size=1, shuffle=False)
chat_hard_loader = torch.utils.data.DataLoader(chat_hard, batch_size=1, shuffle=False)
reasoning_loader = torch.utils.data.DataLoader(reasoning, batch_size=1, shuffle=False)
safety_loader = torch.utils.data.DataLoader(safety, batch_size=1, shuffle=False)

In [7]:
real_tokenizer = AutoTokenizer.from_pretrained(tokenizer)

In [8]:
import tqdm
from tqdm import tqdm

In [29]:
def eval_dataset(model, subject_dataloader):
    total = 0
    correct = 0

    model.to("cuda")
    model.eval()

    with torch.no_grad():
        for example in tqdm(subject_dataloader):
            input_ids = example['input_ids'].clone()
            input_ids[:, -2] = real_tokenizer.mask_token_id
            input_ids = input_ids.to("cuda")
            attention_mask = example['attention_mask'].to("cuda")


            output = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = output.logits
            pred = torch.argmax(logits, dim=-1)

            # print(output)
            print(real_tokenizer.decode(pred[0]))

            # print(real_tokenizer.decode(input_ids[0]))
            if pred[0, -2] == example['input_ids'][0, -2]:
                correct += 1
            total += 1

            
    print(f"Accuracy: {correct / total}")
    return correct / total

In [30]:
# chat_acc = eval_dataset(model, chat_loader)
# chat_hard_acc = eval_dataset(model, chat_hard_loader)
reasoning_acc = eval_dataset(model, reasoning_loader)
# safety_acc = eval_dataset(model, safety_loader)

  0%|          | 0/1431 [00:00<?, ?it/s]

  0%|          | 4/1431 [00:00<01:18, 18.08it/s]

[CLS]Determine which response is the best choice based on mathematical or programming logic.  Choice 0: Write a Python function `has_close_elements(numbers: List[float], threshold: float) -> bool` to solve the following problem:
Check if in given list of numbers, are any two numbers closer to each other than
given threshold.
>>> has_close_elements([1.0, 2.0, 3.0], 0.5)
False
>>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
True
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx!= idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False
# Choice 1: Write a Python function `has_close_elements(numbers: List[float], threshold: float) -> bool` to solve the following problem:
Check if in given list of numbers, are any two numbers closer to each other than
given threshold.
>>> has_close_elements([1.0, 2.0, 3.0], 0.5)
False
>>> has_close_elemen

  1%|          | 8/1431 [00:00<01:19, 17.91it/s]

[CLS]Determine which response is the best choice based on mathematical or programming accuracy.  Choice 0: Write a Python function `mean_absolute_deviation(numbers: List[float]) -> float` to solve the following problem:
For a given list of input numbers, calculate Mean Absolute Deviation
around the mean of this dataset.
Mean Absolute Deviation is the average absolute difference between each
element and a centerpoint (mean in this case):
MAD = average | x - x_mean |
>>> mean_absolute_deviation([1.0, 2.0, 3.0, 4.0])
1.0
    mean = sum(numbers) / len(numbers)
    return sum(abs(x - mean) for x in numbers) / len(numbers)
# Choice 1: Write a Python function `mean_absolute_deviation(numbers: List[float]) -> float` to solve the following problem:
For a given[SEP] of input numbers, calculate Mean Absolute Deviation
around the mean of this dataset.
Mean Absolute Deviation is the average absolute difference between each
element and a centerpoint (mean in this case):
MAD = average | x - x_mean |


...

  8%|▊         | 116/1431 [00:06<01:15, 17.41it/s]

[CLS]Determine which response is the best choice based on mathematical or programming logic.  Choice 0: Write a Python function `reverse_delete(s,c)` to solve the following problem:
Task
We are given two strings s and c, you have to deleted all the characters in s that are equal to any character in c
then check if the result string is palindrome.
A string is called palindrome if it reads the same backward as forward.
You should return a tuple containing the result string and True/False for the check.
Example
For s = "abcde", c = "ae",the result should be ('bcd',False)
For s = "abcdef", c = "b"  the result should be ('acdef',False)
For s = "abcdedcba", c = "ab",the result should be ('cdedc',True)
    s = ''.join([char for char in s if char not in c])
    return (s,s[::-1] == s)
Solution Choice 1: Write a Python function `reverse_delete(s,c)` to solve the following problem:
Task
We are given two strings s and c,[SEP] have to deleted all the characters in s that are equal to any character

  8%|▊         | 119/1431 [00:06<01:14, 17.57it/s]


[CLS]Determine which response is the best choice based on mathematical or programming accuracy.  Choice 0: Write a Python function `sort_array(arr)` to solve the following problem:
In this Kata, you have to sort an array of non-negative integers according to
number of ones in their binary representation in ascending order.
For similar number of ones, sort based on decimal value.
It must be implemented like this:
>>> sort_array([1, 5, 2, 3, 4]) == [1, 2, 3, 4, 5]
>>> sort_array([-2, -3, -4, -5, -6]) == [-6, -5, -4, -3, -2]
>>> sort_array([1, 0, 2, 3, 4]) [0, 1, 2, 3, 4]
    return sorted(sorted(arr), key=lambda x: bin(x)[2:].count('1'))
# Choice 1: Write a Python function `sort_array(arr)` to solve the following problem:
In this Kata,[SEP] has to sort an array of non-negative integers according to
number of ones in their binary representation in ascending order.
For similar number of ones, sort based on decimal value.
It must be implemented like this:
>>>sort_array([1, 5, 2, 3, 4]) == [

KeyboardInterrupt: 

In [67]:
final_results = {
    "chat": chat_acc,
    "chat_hard": chat_hard_acc,
    "reasoning": reasoning_acc,
    "safety": safety_acc
}

print(final_results)

{'chat': 0.7946141479099679, 'chat_hard': 0.2521551724137931, 'reasoning': 0.76659678546471, 'safety': 0.7918918918918919}


In [ ]:
# for chat
0.8657556270096463 * (1 - 0.157) + 0.36853448275862066 * (0.157)

0.7876919073622352

In [ ]:
# for reasoning
0.3305380852550664